### Divide and Conqure problem:



#### 1) Find the closest pair of points in a plane

In [1]:
def dist(a, b):
    return (((a[0]-b[0])**2) +((a[1]-b[1])**2))**0.5

In [2]:
def closest_brute_force(points):
    min_dist = float("inf")
    p1 = None
    p2 = None
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            d = dist(points[i], points[j])
            if d < min_dist:
                min_dist = d
                p1 = points[i]
                p2 = points[j]
    return p1, p2, min_dist

In [3]:
def closest(points):
    x_sorted = sorted(points, key=lambda point: point[0])
    y_sorted = sorted(points, key=lambda point: point[1])
    
    def _recursive_find(x, y):
        n = len(x)
        if n<=3:
            return closest_brute_force(x) # base case --> constant time. 
        else:
            midpoint = x[n//2]
            x_left = x[:n//2]
            x_right = x[n//2:]
            y_left = []
            y_right = []
            for point in y: # send left points and right points to their y_left, y_right lists
                y_left.append(point) if (point[0]<=midpoint[0]) else y_left.append(point)
                
            (p1_left, p2_left, d_l) = _recursive_find(x_left, y_left) # search in left half --> T(n/2)
            (p1_right, p2_right, d_r) = _recursive_find(x_right, y_right) # search in right half --> T(n/2)
            
            (p1, p2, d) = (p1_left, p2_left, d_l) if (d_l<d_r) else (p1_right, p2_right, d_r)
            
            # for (a, b) in left and right --> need to search in [-d, d] window around midpoint
            # reducing search space in x direction 
            reduced_in_x = [point for point in y if midpoint[0] - d < point[0] < midpoint[0]+d]
            
            for i in range(len(reduced_in_x)): # search for every point --> O(n)
                # reducing search space in y direction --> need to search only in next 7 points (proof..)
                for j in range(i+1, min(i+7, len(reduced_in_x))): # constant time --> O(1) step
                    d_new = dist(reduced_in_x[i], reduced_in_x[j])
                    if d_new < d:
                        (p1, p2, d) = (reduced_in_x[i], reduced_in_x[j], d_new)
        return p1, p2, d
    
    return _recursive_find(x_sorted, y_sorted)

In [4]:
test_points = [(2,3), (12, 30), (40, 50), (5,1), (12,10), (3,4)]

In [5]:
print(closest(test_points))

((2, 3), (3, 4), 1.4142135623730951)


#### 2)  Phone Number Mnemonics
Given a string digits representing a phone number, return all possible character arrangements that can result from the number.

A numbered telephone keypad looks like so:
- 2 -> "a" || "b" || "c"
- 3 -> "d" || "e" || "f"
- 4 -> "g" || "h" || "i"
- 5 -> "j" || "k" || "l"
- 6 -> "m" || "n" || "o"
- 7 -> "p" || "q" || "r" || "s"
- 8 -> "t" || "u" || "v"
- 9 -> "w" || "x" || "y" || "z"

***Constraints:***
* s will only digits between 2 and 9. 
* 2 <= n <= 10 (constraint on length of string). 

In [16]:
def get_arrangements(digits):
    solution = []
    digit2chars = {'2':'abc', '3':'def', '4':'ghi', '5':'jkl', '6':'mno', '7':'pqrs', '8':'tuv', '9':'wxyz'}
    def _recursive_build(current_str, remaining_digits):
        if len(remaining_digits) == 0:
            solution.append(current_str)
            return 
        for char in digit2chars[remaining_digits[0]]:
            _recursive_build(current_str+char, remaining_digits[1:])
    _recursive_build(current_str='', remaining_digits=digits)
    return solution

In [17]:
Input =  "43"
Output =  ["gd","ge","gf","hd","he","hf","id","ie","if"]

print(get_arrangements(Input))

['gd', 'ge', 'gf', 'hd', 'he', 'hf', 'id', 'ie', 'if']


#### 3) IP Address Restoration

Given a "raw" ip address string s, return all "restored" ip address strings that can be recovered from s.

A "raw" ip address string is a string of digits that can have . marks inserted to create a valid ip address.

***Constraints:*** 
- The raw ip string will only have digits 1 to 9
- 4 <= n <= 12


In [84]:
ip = '255.255.255.255'
org = '255255255255'

In [85]:
ip.split(".")

['255', '255', '255', '255']

In [99]:
def is_valid(ip, original):
    splitted = ip.split(".")
    for section in splitted:
        if len(section) >3:
            return False
        if len(section) < 1:
            return False 
        if int(section)>255:
            return False
    combined = ''.join([section for section in splitted])
    if combined != original:
        return False 
    return True 

In [100]:
is_valid(ip, org)

True

In [112]:
def split_at_idx(string, idx):
    try:
        a = string[0:idx]
    except:
        a = ''
    try: 
        b = string[idx:]
    except:
        b = ''
    return a, b

In [131]:
def restore_ip_address(ip_address):
    solution = set()
    def _recursive_build(processed, remaining, depth):
        if depth == 4:
            if is_valid(processed[1:], ip_address):
                solution.add(processed[1:])
            return
        _recursive_build(processed+'.'+ split_at_idx(remaining, 1)[0], split_at_idx(remaining, 1)[1],depth+1)
        _recursive_build(processed+'.'+ split_at_idx(remaining, 2)[0], split_at_idx(remaining, 2)[1],depth+1)
        _recursive_build(processed+'.'+ split_at_idx(remaining, 3)[0], split_at_idx(remaining, 3)[1],depth+1)
    
    _recursive_build('', ip_address, 0)
    return [item for item in solution]

In [132]:
Input= "255255232132"
Output= ["255.255.232.132"]
    
restore_ip_address(Input)

['255.255.232.132']

In [133]:
Input= "125523213"
Output=[
  "1.255.23.213",
  "1.255.232.13",
  "12.55.23.213",
  "12.55.232.13",
  "125.5.23.213",
  "125.5.232.13",
  "125.52.3.213",
  "125.52.32.13"]

restore_ip_address(Input)

['125.52.32.13',
 '1.255.232.13',
 '1.255.23.213',
 '125.52.3.213',
 '12.55.23.213',
 '125.5.23.213',
 '12.55.232.13',
 '125.5.232.13']